# Covid no Estado do Ceará

https://www.youtube.com/watch?v=rckzwVBe7gk&t=3255s

In [1]:
# importando as bibliotecas necessárias

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import folium
from folium import plugins
import datetime as dt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

* Data from covid: https://brasil.io/home/
* Data of the cities: https://github.com/kelvins/Municipios-Brasileiros

In [2]:
url = 'https://brasil.io/dataset/covid19/caso_full/?state=CE&format=csv'
data_covid = pd.read_csv(url, parse_dates=True)
url = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/master/csv/municipios.csv'
cities_lat_long = pd.read_csv(url, parse_dates=True)

In [3]:
cities_lat_long.head(2)

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,0,31


In [4]:
data_covid.head(2)

,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population_2019,is_last,is_repeated
0,23,2020-06-06,41,CE,Abaiara,2300101.0,city,6,51.12039,0,1,0,0.1667,11737.0,True,False
1,23,2020-06-06,50,CE,Acarape,2300150.0,city,103,689.93235,0,6,0,0.0583,14929.0,True,False


In [5]:
data = data_covid.merge(cities_lat_long,
                        how = 'inner',
                        left_on='city_ibge_code', 
                        right_on='codigo_ibge')
data.columns

Index(['epidemiological_week', 'date', 'order_for_place', 'state', 'city',
       'city_ibge_code', 'place_type', 'last_available_confirmed',
       'last_available_confirmed_per_100k_inhabitants', 'new_confirmed',
       'last_available_deaths', 'new_deaths', 'last_available_death_rate',
       'estimated_population_2019', 'is_last', 'is_repeated', 'codigo_ibge',
       'nome', 'latitude', 'longitude', 'capital', 'codigo_uf'],
      dtype='object')

In [6]:
data.drop(['order_for_place', 'place_type', 'is_repeated', 'codigo_ibge', 'nome'], axis = 1, inplace=True)

In [7]:
data['city'].replace('Itapajé', 'Itapagé', inplace = True)

In [8]:
data.columns

Index(['epidemiological_week', 'date', 'state', 'city', 'city_ibge_code',
       'last_available_confirmed',
       'last_available_confirmed_per_100k_inhabitants', 'new_confirmed',
       'last_available_deaths', 'new_deaths', 'last_available_death_rate',
       'estimated_population_2019', 'is_last', 'latitude', 'longitude',
       'capital', 'codigo_uf'],
      dtype='object')

Abaixo eu crio um DataFrame com o código do iBGE e a respectiva cidade, e em seguida eu junto com os dados anteriores.
Fiz isso porque ao tentar plotar as informações o gráfico ficava todo preto, porque por algum motivo o Código do IGE do arquivo Json e do DataFrame fornecido pelo brasil.io não eram iguais. 

No arquivo Json o Código do IBGE é uma string, e por mais que eu convertesse os valores do DataFrame pra string ainda assim aparentemente eles não eram iguais(mesmo eu claramente vendo que eram iguais). Comprovei isso ao tentar fazer inner join nos dois datasets pelo Código do IBGE e o retorno era um DataFrame vazio. Quando tentei fazer o mesmo com as cidades o resultado funcionou.

Então decidi fazer inner join com os nomes das cidades e excluir a coluna do Código do IBGE do DataFrame inicial, e a partir de então utilizar o Código do IBGE que extraí do arquivo Json, que funcionou.

Outros pontos que foram corrigidos:
* O município do Itapagé está escrito como 'Itapajé' no data set do brasil.io ;
* O município do granjeiro não tem casos confirmados, entao no dataset que peguei do Brasil IO não cosnta as informações da Cidade;

In [9]:
import json

ce_cities = json.load(open('data/geojs-23-mun.json'))

id_cidade = []
nome_cidade = []

for i in range(len(ce_cities['features'])):
    Id = ce_cities['features'][i]['properties']['id']
    id_cidade.append(Id)
    nome = ce_cities['features'][i]['properties']['name']
    nome_cidade.append(nome)

ce = pd.DataFrame({'ibge_code': id_cidade, 'city': nome_cidade})

In [10]:
data = data.merge(ce, how = 'outer').drop('city_ibge_code', axis = 1)

# Mapa Confirmados

* Geodata Ceará on: https://github.com/tbrugz/geodata-br

In [11]:
columns = ['ibge_code', 'city', 'last_available_confirmed_per_100k_inhabitants']
current_data = data[data['is_last'].fillna(True)][columns].fillna(0)

In [12]:
ceara = folium.Map( width= '100%',
                   tiles='Mapbox Bright',
                    height = '100%',
                    location = [-5.15, -39.5],
                   zoom_control=False,
                   min_zoom=7, max_zoom=9,
                   min_lat=-2.15, max_lat=-8.15, 
                   min_lon=-37, max_lon=-42, 
                   max_bounds=True,
                    zoom_start = 7)

In [13]:
folium.Choropleth(geo_data = ce_cities,
                  name = 'Casos Confirmados',
                  data = current_data,
                  columns = ['ibge_code' ,'last_available_confirmed_per_100k_inhabitants'],
                  key_on = 'feature.properties.id', 
                  fill_color = 'YlOrRd',
                  fill_opacity = 0.6,
                  line_color = 'black',
                  line_opacity = 1,
                  line_weight=0.25,
                  legend_name = 'Confirmed cases per 100k hab.',
                  show = True,
                 ).add_to(ceara)


In [14]:
ceara

In [16]:
ceara.save('maps/ceara_cases_per_100k_hab.html')